# WildWing telemetry data analysis
### Purpose: Script to analyze and visualize telemetry data produced by WildWing

### Step 1: Import libraries

In [ ]:
import pandas as pd
import geopy
from geopy import distance
import folium
from folium import IFrame
# import pyproj
import base64
import os
import cv2
import branca

### Step 2: Define functions to process telemetry

In [ ]:
def get_bearing(lat1, long1, lat2, long2):
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(long1, lat1, long2, lat2)
    return fwd_azimuth

In [ ]:
def get_distance(lat1, long1, lat2, long2):
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(long1, lat1, long2, lat2)
    return distance

In [ ]:
def get_speed(distance, time1, time2):
    if distance == 0:
        return 0
    else:
        geodesic = pyproj.Geod(ellps='WGS84')
        time_diff = (time2 - time1).total_seconds()
        speed = distance/time_diff
        return speed

### Step 3: Define sessions to process

In [ ]:
sessions = ['zebra_073124_122452', 'giraffe_073124_141828', 'phorses_073124_144555']

### Step 4: Process telemetry data

Use functions to calculate bearing (a.k.a. heading) of the drone, distance travelled, and speed. Save csv telemetry file.

In [ ]:
# create telemetry csv files
telemetry = []
for session in sessions:
    df = pd.read_csv(f'wildwingdeployment/{session}/telemetry_log.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['latitude'] = df['x'].astype(float)
    df['longitude'] = df['y'].astype(float)
    df['directory'] = f'wildwingdeployment/{session}/'
    df.drop_duplicates(subset=['timestamp'], keep='first', inplace=True)  # remove duplicate timestamps
    df.reset_index(drop=True, inplace=True) # reset index
    for i in range(1, len(df)):
        df.at[i, 'distance (m)'] = get_distance(df.at[i-1, 'latitude'], df.at[i-1, 'longitude'], df.at[i, 'latitude'], df.at[i, 'longitude'])  # distance in meters
        df.at[i, 'bearing (deg)'] = get_bearing(df.at[i-1, 'latitude'], df.at[i-1, 'longitude'], df.at[i, 'latitude'], df.at[i, 'longitude'])  # bearing in degrees
        df.at[i, 'speed (m/s)'] = get_speed(df.at[i, 'distance (m)'], df.at[i-1, 'timestamp'], df.at[i, 'timestamp'])
    df.to_csv(f'telemetry/{session}_telemetry.csv', index=False)
    telemetry.append(df)

Print statistics on telemetry files

In [ ]:
# concat all telemetry data
telemetry_df = pd.concat(telemetry)
telemetry_df.reset_index(drop=True, inplace=True)
telemetry_df[['latitude', 'longitude', 'distance (m)','bearing (deg)','speed (m/s)']].describe()

In [ ]:
# describe telemetry data as latex table
table = telemetry_df[['latitude', 'longitude', 'distance (m)','bearing (deg)','speed (m/s)']].describe().to_latex()
print(table)

In [ ]:
# get the duration of each session
for df in telemetry:
    start = df['timestamp'].iloc[0]
    end = df['timestamp'].iloc[-1]
    duration = (end - start).total_seconds()
    session = df['directory'].iloc[0].split('/')[1]
    print(f'{session} duration: {duration}')

### Step 5: Map the telemetry data 

Crop frames to preview on the map

In [ ]:
for s in sessions:
    frames = os.listdir('wildwingdeployment/'+s+'/frames/')
    for f in frames:
        # load image
        img = cv2.imread('wildwingdeployment/'+s+'/frames/'+f)
        if img is not None:
            # trim image
            crop_img = img[260:460, 440:840]
            # save image
            cv2.imwrite('wildwingdeployment/'+s+'/cropped_frames/'+f, crop_img)


In [ ]:
# define colors based on speed of drone
def speed_color(speed):
    if speed < 0.05:
        return 'green'
    elif speed < 0.4:
        return 'yellow'
    else:
        return 'red'

In [ ]:
# Function to create a legend
def create_legend():
    legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 250px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Start <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; End <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:gray"></i>&nbsp; Photo <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Bearing <br>
     &nbsp; <i class="fa fa-circle fa-2x" style="color:green"></i>&nbsp; Slow <br>
     &nbsp; <i class="fa fa-circle fa-2x" style="color:yellow"></i>&nbsp; Medium <br>
     &nbsp; <i class="fa fa-circle fa-2x" style="color:red"></i>&nbsp; Fast <br>
     </div>
     '''
    return legend_html

# Create a Folium map centered at an average location
m = folium.Map(location=[lat, long], zoom_start=20)

# Super group for all traces, photos, and bearings
super_speed = folium.FeatureGroup(name='Speed', overlay=True)
super_photo = folium.FeatureGroup(name='Photo Preview', overlay=True)
super_bearing = folium.FeatureGroup(name='Bearing', overlay=True)

# Add each trace to the map
for trace in telemetry:
    # Create a feature group for each trace
    name = trace['directory'].iloc[0].split('/')[1]
    fg = folium.FeatureGroup(name=name, overlay=True)
    
    # Plot start and end points
    start = trace.iloc[0]
    folium.Marker([start['latitude'], start['longitude']], popup='Start', icon=folium.Icon(color='green')).add_to(fg)
    end = trace.iloc[-1]
    folium.Marker([end['latitude'], end['longitude']], popup='End', icon=folium.Icon(color='red')).add_to(fg)
    folium.PolyLine(trace[['latitude', 'longitude']].values).add_to(fg)
    fg.add_to(m) # Add the feature group to the map
    
    # Add a feature group for photos
    fg_photo = folium.FeatureGroup(name=f'{name} - Photos', overlay=True)
    for index, row in trace.iterrows():
        index += 20
        if index % 40 == 0:
            # get the frame_directory
            frame_directory = row['directory']
            file = frame_directory + 'cropped_frames/' + str(index) + '.jpg'
            
            # Add Marker
            encoded = base64.b64encode(open(file, 'rb').read())
            html = '<img src="data:image/png;base64,{}">'.format
            iframe = IFrame(html(encoded.decode('UTF-8')), width=400, height=350)
            popup = folium.Popup(iframe, max_width=400)

            folium.Marker(location=[row['x'], row['y']], tooltip=html, popup=popup, 
                          icon=folium.Icon(color='gray')).add_to(fg_photo)
    #fg_photo.add_to(m)
    fg_photo.add_to(super_photo)
    
    # Add a feature group for speed
    speed_fg = folium.FeatureGroup(name=f'{name} - Speed', overlay=True)
    for index, row in trace.iterrows():
        speed = row['speed (m/s)']
        color = speed_color(speed)
        folium.CircleMarker([row['latitude'], row['longitude']], radius=5, color=color).add_to(speed_fg)
    # speed_fg.add_to(m)
    speed_fg.add_to(super_speed)
    
    # Add a feature group for bearing
    bearing_fg = folium.FeatureGroup(name=f'{name} - Bearing', overlay=True)
    for index, row in trace.iterrows():
        index += 20
        if index % 30 == 0:
            angle = row['bearing (deg)']
            icon = folium.Icon(color='blue', icon='arrow-up', prefix='fa', icon_angle=angle)
            folium.Marker([row['latitude'], row['longitude']], icon=icon, tooltip=str(angle)).add_to(bearing_fg)
    # bearing_fg.add_to(m)
    bearing_fg.add_to(super_bearing)

# Add super groups to the map
super_speed.add_to(m)
super_photo.add_to(m)
super_bearing.add_to(m)

# Add layer control to toggle traces
folium.LayerControl().add_to(m)

# Add legend to the map
legend_html = create_legend()
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save('map.html')

# Display the map
m